## Pip & Import

In [ ]:
!pip install -q seqeval==1.2.2 transformers==4.28.0 datasets==2.14.5 sentencepiece==0.1.99 accelerate==0.22.0

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
# from datasets import load_dataset, load_metric, Dataset, DatasetDict
import json
import itertools
import numpy as np
import logging
import torch
import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Annotated Data

In [ ]:
with open("/content/Train.json") as src:
    ner_annotated_data = json.loads(src.read())

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding=True ,max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def compute_metrics(p):
    global model_name, current_epoch

    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [custom_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [custom_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    metric_results = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

    return metric_results

def compute_results(trainer, tokenized_ds, metric, custom_labels):
    predictions, labels, _ = trainer.predict(tokenized_ds)
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [custom_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [custom_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results

In [ ]:
all_ner_tags, all_ner_tokens = [], []
o_tag = "O"

for rec in ner_annotated_data:
    ner_tags, ner_tokens = [], []

    text = rec["data"]["text"]
    if len(rec["annotations"]) == 0 or "result" not in rec["annotations"][0] or len(rec["annotations"][0]["result"]) == 0:
        continue

    # sort rec["annotations"][0]["result"] based on start index
    rec["annotations"][0]["result"].sort(key=lambda x: x["value"]["start"])

    # collect all ranges with their labels
    ranges = []
    for r in rec["annotations"][0]["result"]:
        ranges.append((range(r["value"]["start"], r["value"]["end"]+1), r["value"]["labels"][0]))

    # split text into tokens
    tokens = text.split()
    token_ranges = []
    c = 0
    for i, token in enumerate(tokens):
        token_ranges.append( (range(c, c+len(token)), token) )
        c += len(token) + 1

    # find all tokens that are in the ranges
    for token_range in token_ranges:
        is_found = False

        for sub_range in ranges:
            if all(e in sub_range[0] for e in token_range[0]):
                ner_tags.append(sub_range[1])
                ner_tokens.append(token_range[1])
                is_found = True
                break

        # not found
        if not is_found:
            ner_tags.append(o_tag)
            ner_tokens.append(token_range[1])

    # format BI prefix
    for i, tag in enumerate(ner_tags):
        if i == 0 and ner_tags[i] != o_tag:
            ner_tags[i] = f"B-{ner_tags[i]}"
            continue

        if i == 0 or ner_tags[i] == o_tag:
            continue

        if ner_tags[i-1].replace("B-","").replace("I-","") == ner_tags[i]:
            ner_tags[i] = f"I-{ner_tags[i]}"
        else:
            ner_tags[i] = f"B-{ner_tags[i]}"


    all_ner_tags.append(ner_tags)
    all_ner_tokens.append(ner_tokens)

In [ ]:
sample_id = 1
for tag, token in zip(all_ner_tags[sample_id], all_ner_tokens[sample_id]):
    print(tag, "--->", token)

O ---> "الحمد
O ---> لله
O ---> والصلاة
O ---> والسلام
O ---> على
O ---> رسول
O ---> ﷲ
O ---> أما
O ---> بعد:فلدى
O ---> الدائرة
O ---> التجارية
O ---> الثانية
O ---> وبناءً
O ---> على
O ---> القضية
O ---> رقم
O ---> ٣٣٢
O ---> لعام
O ---> ١٤٤٢
O ---> هـالمقامة
O ---> من/
O ---> شركة
B-ORG ---> الشرق
I-ORG ---> للخرسانة
I-ORG ---> الجاهزة
O ---> سجل
O ---> تجاري
O ---> (...)
O ---> ضد/
O ---> شركة
B-ORG ---> درة
I-ORG ---> الغمام
I-ORG ---> للتجارة
I-ORG ---> والمقاولات
O ---> غير
O ---> ذلك
O ---> (...)
O ---> القاضي
O ---> عبدالرحمن
O ---> بن
O ---> فايز
O ---> الفايز
O ---> رئيسا
O ---> (الوقائع)توجز
O ---> بأن
O ---> وكيل
O ---> المدعية
O ---> تقدمت
O ---> بصحيفة
O ---> دعوى
O ---> ورد
O ---> فيها
O ---> (
O ---> تم
O ---> التعاقد
O ---> فيما
O ---> بين
O ---> موكلتنا
O ---> المدعية
O ---> والمدعى
O ---> عليها
O ---> موكلتنا
O ---> لتوريد
O ---> كمية
O ---> من
O ---> الخرسانة
O ---> الجاهزة
O ---> بحجم
O ---> ٨٣٠٠
O ---> متر
O ---> لإنشاء
O ---> مكتب
O ---> العمل
O ---> الثاني
O --

In [ ]:
len(all_ner_tags)

100

In [ ]:
train_texts = all_ner_tokens[:80]
train_tags = all_ner_tags[:80]

dev_texts = all_ner_tokens[80:]
dev_tags = all_ner_tags[80:]

In [ ]:
set(itertools.chain.from_iterable(all_ner_tags))

{'B-DATE', 'B-NUM', 'B-ORG', 'B-PER', 'I-DATE', 'I-NUM', 'I-ORG', 'I-PER', 'O'}

In [ ]:
# marefa-ner base checkpoint
base_checkpoint = "marefa-nlp/marefa-ner"
task = "ner"
label_all_tokens = True
seed = 101

# where to save the new model and its logs
new_model_path = f"./finetuned-ner"
logs_path = f"./logs"

# seqeval metric
metric = load_metric("seqeval")

## all of the tags in your dataset
custom_labels = ['B-DATE', 'B-NUM', 'B-ORG', 'B-PER', 'I-DATE', 'I-NUM', 'I-ORG', 'I-PER', 'O']

device = "cuda:0"

In [ ]:
## convert to Dataset
datasets = DatasetDict({
    "train": Dataset.from_dict({
        "tokens": train_texts,
        "ner_tags": [ [ custom_labels.index(r) for r in rec ] for rec in train_tags ]
    }),
    "dev": Dataset.from_dict({
        "tokens": dev_texts,
        "ner_tags": [ [ custom_labels.index(r) for r in rec ] for rec in dev_tags ]
    }),
})

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 80
    })
    dev: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 20
    })
})

## Fine-Tuning

In [ ]:
from transformers import set_seed

set_seed(seed)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(base_checkpoint,
                                                        num_labels=len(custom_labels),
                                                        ignore_mismatched_sizes=True).to(device)

In [ ]:
# prepare dataset
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

In [ ]:
# configure your fine-tuning process

args = TrainingArguments(
    new_model_path,
    logging_dir=logs_path,
    evaluation_strategy = "epoch",
    logging_strategy= "epoch",
    save_strategy= "no",
    learning_rate= 1e-4,
    load_best_model_at_end= False,
    per_device_train_batch_size= 4,
    per_device_eval_batch_size= 4,
    num_train_epochs= 100,
    weight_decay= 0.01,
    push_to_hub= False,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.677400,0.181729,0.014085,0.005051,0.007435,0.954706
2,0.299500,0.217066,0.170782,0.419192,0.242690,0.929706
3,0.269100,0.141849,0.262136,0.272727,0.267327,0.957059
4,0.234300,0.168710,0.378049,0.469697,0.418919,0.963529
5,0.194000,0.157082,0.329787,0.313131,0.321244,0.953627
6,0.221200,0.204761,0.162218,0.398990,0.230657,0.922059
7,0.185600,0.128079,0.349282,0.368687,0.358722,0.965098
8,0.163100,0.131140,0.407407,0.333333,0.366667,0.965392
9,0.153800,0.154112,0.340000,0.429293,0.379464,0.958529
10,0.133500,0.126412,0.330144,0.348485,0.339066,0.962059


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
## evaluate the model
dev_results = compute_results(trainer, tokenized_datasets["dev"], metric, custom_labels)

In [ ]:
dev_results

{'DATE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 12},
 'NUM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'ORG': {'precision': 0.4166666666666667,
  'recall': 0.46511627906976744,
  'f1': 0.43956043956043955,
  'number': 43},
 'PER': {'precision': 0.3614457831325301,
  'recall': 0.4195804195804196,
  'f1': 0.38834951456310673,
  'number': 143},
 'overall_precision': 0.22346368715083798,
 'overall_recall': 0.40404040404040403,
 'overall_f1': 0.28776978417266186,
 'overall_accuracy': 0.9373529411764706}

#### Save to Your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p /gdrive/MyDrive/finetuned-ner-model-t5

In [ ]:
new_model_path = "/content/drive/MyDrive/finetuned-ner-model-t5"

trainer.save_model(f"{new_model_path}/best")
tokenizer.add_tokens(custom_labels)
tokenizer.save_pretrained(f"{new_model_path}/best")

('/content/drive/MyDrive/finetuned-ner-model-t5/best/tokenizer_config.json',
 '/content/drive/MyDrive/finetuned-ner-model-t5/best/special_tokens_map.json',
 '/content/drive/MyDrive/finetuned-ner-model-t5/best/tokenizer.json')

## RegEx

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/github_data')

In [ ]:
df = df[df["judgment_text"].notnull()]

In [ ]:
# df['judgment_regex']=df['judgment_text'].apply(lambda x: re.sub(r'\s+', ' ', x))
# df['judgment_regex']=df['judgment_regex'].apply(lambda x: re.sub(r"[)(><.:',،`]", ' ', x))

In [ ]:
def process_text(text):
        if "ضد/" in text:
            match = re.search(r'ضد/\s*(شركة|مؤسسة|مصنع)\s*', text)
            if match:
                text = re.sub(r'ضد/', '', text)
            else:
                text = re.sub(r'ضد/\s*\w+\s*', '', text)

        if "من/" in text:
            match = re.search(r'من/\s*(شركة|مؤسسة|مصنع)\s*', text)
            if match:
                text = re.sub(r'من/', '', text)
            else:
                text = re.sub(r'(ضد|من)/\s*\w+(\s+\w+){0,3}\s*', '', text)
        words = text.split()
        for i in range(min(4, len(words))):
            words.pop(0)

        return ' '.join(words)

df['judgment_regex'] = df['judgment_text'].apply(process_text)

In [ ]:
  # to_remove = ["الحمد لله" ,  "والصلاة"," والسلام ","على رسول ﷲ" , "القاضي" , "رئيسا" , "عضوا" , "وبناءً القضية" , "رئيس الدائرة" ,
  # "وصلى ﷲ وسلم نبينا محمد وآله وصحبه أجمعين"
  # , "وبﷲ التوفيق و" , "أما بعد" ,"سجل تجاري", "سجل تجاري رقم" , "سجل مدني رقم" , "للمقاولات" , "المقاولات" , "الديكور الداخلي", "المحدودة"]

  # # for sentence in to_remove:
  #   df['judgment_regex'] = df['judgment_regex'].str.replace(sentence, '')

In [ ]:
def remove_text(text):
    if " بن " in text:
        text = re.sub(r'\S+\sبن\b', 'بن', text)
        match = re.search(r'بن\s+(\S+)\s+(\S+)\s*', text)
        if match:
            word1 = match.group(1)
            word2 = match.group(2)
            if word2 == "بن":
                text = re.sub(r'\S+\sبن\b', '', text)
                next_words_match = re.search(r'بن\s+(\S+)\s+(\S+)\s+(\S+)\s*', text)
                if next_words_match:
                    next_word3 = next_words_match.group(2)
                    next_word4 = next_words_match.group(3)
                    if next_word4.startswith('ال'):
                        return re.sub(r'بن\s+(\S+)\s+(\S+)\s*', '', text)
                    elif not word2.startswith('ال'):
                          return re.sub(r'بن\s+\S+\s*', '', text, count=1)
                return 1+1
            elif word2.startswith('ال'):
                return re.sub(r'بن\s+\S+\s+\S+\s*', '', text)
                return 1+1
            elif not word2.startswith('ال'):
                return re.sub(r'بن\s+\S+\s*', '', text, count=1)
    return text

In [ ]:
df['judgment_regex'] = df['judgment_regex'].apply(remove_text)

In [ ]:
df['judgment_regex'][0]

'على رسول ﷲ أما بعد:فلدى الدائرة التجارية الثامنة وبناءً على القضية رقم ٥٤٨٣ لعام ١٤٤٢ هـالمقامة سجل تجاري (...) شركة ديبا العربية السعودية للمقاولات والديكور الداخلي المحدودة غير ذلك (...) القاضي رئيسا(الوقائع)تتلخص وقائع هذه القضية بالقدر اللازم للحكم فيها في أنّه تقدّم وكيل المدعي بصحيفة دعوى إلى المحكمة التجارية بالرياض جاء فيها مطالبة المدعى عليها بسداد مبلغ وقدره (١٢٩.٤٥٧.٣) مائة وتسعة وعشرون ألف وأربعمائة وسبعة وخمسون ريالا وثلاثة هللات نظير قيام المدعي بتنفيذ أعمال تركيب مشغولات خشبية وأبواب في مشروع فندق موطن لقاء ولم تقم بسداد المتبقي من قيمة تلك الأعمال، وبإحالة الأوراق تم قيد الدعوى قضية بالرقم المشار إليه أعلاه وأحيلت إلى هذه الدائرة والتي باشرت نظرها وفقا لما هو مدون في محاضرها، ففي جلسة ١١/١٠/١٤٤٢هـ حضر وكيل المدعي فيما تبين عدم حضور من يمثل المدعى عليها رغم تبلغهم بموعد ورابط هذه الجلسة، وقد قررت الدائرة سماع الدعوى وبسؤال وكيل المدعي عن الدعوى أحال إلى صحيفة الدعوى وبسؤاله عن بينات موكله طلب مهلة لذلك وأفهمته الدائرة بضرورة تغطيتها بمذكرة شارحة وإيداعها عبر الموقع الإل

## Remove NER form text

In [ ]:
def _extract_ner(text: str, model: AutoModelForTokenClassification,
                 tokenizer: AutoTokenizer, start_token: str="▁"):
    tokenized_sentence = tokenizer([text], padding=True, truncation=True, return_tensors="pt")
    tokenized_sentences = tokenized_sentence['input_ids'].numpy()

    with torch.no_grad():
        output = model(**tokenized_sentence.to("cuda:0"))

    last_hidden_states = output[0].cpu().numpy()
    label_indices = np.argmax(last_hidden_states[0], axis=1)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentences[0])
    special_tags = set(tokenizer.special_tokens_map.values())

    grouped_tokens = []
    for token, label_idx in zip(tokens, label_indices):
        if token not in special_tags:
            if not token.startswith(start_token) and len(token.replace(start_token,"").strip()) > 0:
                grouped_tokens[-1]["token"] += token
            else:
                grouped_tokens.append({"token": token, "label": custom_labels[label_idx]})

    # extract entities
    ents = []
    prev_label = "O"
    for token in grouped_tokens:
        label = token["label"].replace("I-","").replace("B-","")
        if token["label"] != "O":

            if label != prev_label:
                ents.append({"token": [token["token"]], "label": label})
            else:
                ents[-1]["token"].append(token["token"])

        prev_label = label

    # group tokens
    ents = [{"token": "".join(rec["token"]).replace(start_token," ").strip(), "label": rec["label"]}  for rec in ents ]

    return ents

In [ ]:
device = "cuda:0"

custom_labels = ['B-DATE', 'B-NUM', 'B-ORG', 'B-PER', 'I-DATE', 'I-NUM', 'I-ORG', 'I-PER', 'O']

model_cp = "/content/drive/MyDrive/ner-model/best"

tokenizer = AutoTokenizer.from_pretrained(model_cp)
model = AutoModelForTokenClassification.from_pretrained(model_cp, num_labels=len(custom_labels)).to(device)

In [ ]:
def remove_names_from_text(text):
    entities = _extract_ner(text=text, model=model, tokenizer=tokenizer)
    # Extract 'PER' tokens
    tokens = [entity['token'] for entity in entities if entity['label'] != 'O']

    for token in tokens:
        # Split the token into words and replace with the first character of each word
        words = token.split()
        # replacement = '.'.join([word[0] for word in words])
        text = text.replace(token, '')

    return text

In [ ]:
df['judgment_regex'] = df['judgment_regex'].astype(str)

In [ ]:
df['judgment_regex'] = df['judgment_regex'].apply(remove_names_from_text)

In [ ]:
df['judgment_regex'][34]

'على رسول ﷲ أما بعد: فلدى دائرة تجارية اولى وبناءً على قضية رقم ٧٧٠١ لعام ١٤٤٢ هـ مقامة ذلك (...) فهد محمد  سعود هوية وطنية (...) قاضي رئيسا قاضي  قاضي  (وقائع) تتحصل وقائع هذه قضية بقدر لازم لإصدار هذا حكم بأن وكيل مدعية تقدم بلائحة دعوى إلى محكمة تجارية برياض جاء فيها ما نصه : "نفيدكم بأنه سبق وأن تعاقدت موكلتي مع مدعى عليه بموجب اتفاقية مؤرخة في ١٧/ ٠٥/ ١٤٣٨ هـ موافق ١٤/ ٠٢/ ٢٠١٧م، لتنفيذ مشروع مسمى بكسب طيب وذي هو عبارة عن استيراد عربات بيع أغذية متجولة، وتسليمها إلى محتاجين ليكون مصدر رزق لهم، على أن تتكفل موكلتي بـ (٧٠%) من تكلفة مشروع ويتحمل مدعى عليه نسبة متبقية ٣٠% وذلك من باب مسؤولية اجتماعية، وبناء على ذلك قامت موكلتي بتسليم مدعى عليه مبلغ قدره (١,٤٥٤,٢٥٢)  وأربعمائة وأربع وخمسون ألفًا ومائتان واثنان وخمسون رياً، إلا أن مدعى عليه لم يستوفي تزاماته مبينة في اتفاقية، إذ قام باستيراد عربات غير مطابقة لاشتراطات أمانة محافظة جدة أمر ذي تعذر معه تحقيق هدف من مشروع، بإضافة إلى عدم وجود أية فواتير معتمدة توضح تكلفة عربات باستثناء بيان جمركي لعدد عشرين عربة يوضح أن قيمتها إجمية تبلغ 

In [ ]:
df.to_csv('/content/drive/MyDrive/dataset/dataset.csv',index=False)